# Задание:

1) выбрать архитектуру диалогового агента (генеративная/ранжирующая)
2) обучить на русскоязычном наборе диалогов (например https://www.kaggle.com/datasets/valentinbiryukov/toloka-persona-chat-rus)
3) опционально добавить возможность персонификации (при желании)

В результате должен получиться инференс диалогового агента, где можно вести диалог (способ ввода на ваше усмотрение, можно просто на инпутах)

In [1]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from torchmetrics.functional.text import bleu_score
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import lightning.pytorch as pl
import pandas as pd
from datasets import load_dataset
from lightning.pytorch.loggers import TensorBoardLogger
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split

In [2]:
import os
os.environ['http_proxy'] = 'http://proxy.stc:3128'
os.environ['https_proxy'] = 'http://proxy.stc:3128'
os.environ['ftp_proxy'] = 'http://proxy.stc:3128'

In [3]:
!nvidia-smi

Tue Dec 12 06:59:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 46%   67C    P2    65W / 250W |   1084MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Преобработка данных

In [4]:
df = pd.read_table('dialogues.tsv')
df.head()

,persona_1_profile,persona_2_profile,dialogue
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...


In [5]:
df['dialogue'][0]

'<span class=participant_2>Пользователь 2: Привет) расскажи о себе</span><br /><span class=participant_1>Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось<br />)</span><br /><span class=participant_2>Пользователь 2: Что читаешь? Мне нравится классика</span><br /><span class=participant_2>Пользователь 2: Я тоже люблю пообщаться</span><br /><span class=participant_1>Пользователь 1: Люблю животных, просто обожаю, как и свою работу)</span><br /><span class=participant_1>Пользователь 1: Я фантастику люблю</span><br /><span class=participant_2>Пользователь 2: А я выращиваю фиалки</span><br /><span class=participant_2>Пользователь 2: И веду здоровый и активный образ жизни!</span><br /><span class=participant_1>Пользователь 1: Ух ты, интересно.</span><br /><span class=participant_2>Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду<br />..</span><br /><span class=participant_1>Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим<br />дру

In [6]:
def process_data(dialog):
    res = []
    #prev = dialog.split('</span>')[0].split(':')[0][-1]
    prev = ''
    #print(prev)
    for sentence in dialog.split('</span>'):
        s = sentence.split(':')
        curr = s[0][-1]
        if curr != prev:
            if curr == '1':
                msg = "<Person1>" + s[1].replace('<br />', '') + ' '
                res.append(msg)
            if curr == '2':
                msg = "<Person2>" + s[1].replace('<br />', '') + ' '
                res.append(msg)
        else:
            msg = s[1].replace('<br />', '') + ' '
            res[-1]+=msg
        prev = curr    
    return res

In [7]:
process_data(df["dialogue"][2])

['<Person1> Привет  Как дела ? ',
 '<Person2> Добрый день!  Хорошо,  чем увлекаетесь? ',
 '<Person1> Я бегаю по утрам а ты?  Есть любимые вещи или еда ?  Занят ? ',
 '<Person2> Я люблю петь в караоке) ',
 '<Person1> Круто ) ',
 '<Person2> Люблю готовить пасту,  у меня классно получается!  Любишь готовить? ',
 '<Person1> Это хорошо  Я не эксперт  Я люблю есть арбуз ']

In [8]:
df["processd_dialog"] = df["dialogue"].map(process_data)

In [9]:
def context(my_list):
    return " ".join(element for element in my_list[:-1])

def answer(my_list):
    return my_list[-1]

In [10]:
df["context"] = df["processd_dialog"].map(context)
df["answer"] = df["processd_dialog"].map(answer)

In [11]:
df["context"][0]

'<Person2> Привет) расскажи о себе  <Person1> Привет) под вкусный кофеек настроение поболтать появилось)  <Person2> Что читаешь? Мне нравится классика  Я тоже люблю пообщаться  <Person1> Люблю животных, просто обожаю, как и свою работу)  Я фантастику люблю  <Person2> А я выращиваю фиалки  И веду здоровый и активный образ жизни!  <Person1> Ух ты, интересно.  <Person2> Ты случайно не принц на белом коне? Я его очень жду.. '

In [12]:
df.head()

,persona_1_profile,persona_2_profile,dialogue,processd_dialog,context,answer
0,<span class=participant_1>У меня любимая работ...,<span class=participant_2>Ищу принца.<br />Вед...,<span class=participant_2>Пользователь 2: Прив...,"[<Person2> Привет) расскажи о себе , <Person1>...",<Person2> Привет) расскажи о себе <Person1> П...,<Person1> А у меня из хобби каждую неделю туси...
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...,"[<Person1> Привет! , <Person2> Привет,Как жизн...","<Person1> Привет! <Person2> Привет,Как жизнь?...","<Person2> Да я надеюсь на это,люблю ее"
2,<span class=participant_1>Я купила дом<br />Я ...,<span class=participant_2>Я пою в караоке<br /...,<span class=participant_1>Пользователь 1: Прив...,"[<Person1> Привет Как дела ? , <Person2> Добр...",<Person1> Привет Как дела ? <Person2> Добрый...,<Person1> Это хорошо Я не эксперт Я люблю ес...
3,<span class=participant_1>я врач и женат<br />...,<span class=participant_2>Я мальчик<br />Я учу...,<span class=participant_2>Пользователь 2: Здра...,"[<Person2> Здравствуйте Я Леша , <Person1> Зд...",<Person2> Здравствуйте Я Леша <Person1> Здра...,<Person2> А... а я на машину... Ого
4,<span class=participant_1>Я школьница.<br />Я ...,<span class=participant_2>Я простоват.<br />Лю...,<span class=participant_1>Пользователь 1: Прив...,"[<Person1> Привет! , <Person2> Привет! Как тв...",<Person1> Привет! <Person2> Привет! Как твои...,<Person2> Поздно уже может до завтра? СПОКОЙНО...


In [13]:
train, test = train_test_split(df, test_size=0.2)

In [14]:
test.to_csv('dialogue_test.csv')  

In [15]:
train.to_csv('dialogue_train.csv')  

In [16]:
df.to_csv('dialogue_dataset.csv')  

In [17]:
data_files = {"train": "dialogue_train.csv", "test": "dialogue_test.csv"}
dataset = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Обучение модели

In [18]:
class GenDataModule(pl.LightningDataModule):
    def __init__(self, model_name_or_path, dataset):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        #self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
        self.ds = dataset
        #self.tokenizer.add_tokens(["<Person1>", "<Person2>"])
        
    def prepare_data(self):
        
        for split in self.ds:
            
            #self.ds[split] = self.ds[split].select(list(range(5000)))
            if split == "test":
                self.ds[split] = self.ds[split].map(self.test_tokenize)
            else:
                self.ds[split] = self.ds[split].map(self.tokenize)
        
    def train_dataloader(self):
        train_split = self.ds["train"]
        return DataLoader(train_split, batch_size=4, collate_fn=self.collate)
    
    def val_dataloader(self):
        val_split = self.ds["test"]
        return DataLoader(val_split, batch_size=4, collate_fn=self.collate)
    
    def test_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=4, collate_fn=self.collate_test)
    
    def predict_dataloader(self):
        pred_split = self.ds["test"]
        return DataLoader(pred_split, batch_size=4, collate_fn=self.collate_test)
    
    def tokenize(self, row):
        row = self.tokenizer(f'{row["context"]} {row["answer"]}', max_length=128, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def test_tokenize(self, row):
        row = self.tokenizer(f'{row["context"]} {row["answer"]}[:9]', max_length=128, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def collate(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]).squeeze() for s in samples])
        attention_mask = torch.stack([torch.tensor(s["attention_mask"]).squeeze() for s in samples])
        
        return {"input_ids": input_ids, "attention_mask": attention_mask}
        

In [19]:
class GenModel(pl.LightningModule):
    def __init__(self, model_name_or_path):
        super().__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
        #self.tokenizer.add_tokens(["<Person1>", "<Person2>"])
    
    def training_step(self, batch, batch_idx):
        
        loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        val_loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("val_loss", val_loss.item(), prog_bar=True)
        return val_loss
    
    def predict_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"], max_new_tokens=200)
        
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        return preds
    
    def infer(self, text):
        inputs = self.tokenizer(text, return_tensors='pt')
        generated_token_ids = self.model.generate(
            **inputs,
            max_new_tokens=40
        )

        context_with_response = [self.tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
        return context_with_response
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=2e-5)
        return optimizer

In [20]:
gen_model = GenModel("ai-forever/rugpt3small_based_on_gpt2")

In [21]:
gen_dm = GenDataModule("ai-forever/rugpt3small_based_on_gpt2", dataset)

In [22]:
trainer = pl.Trainer(max_epochs=3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/8010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2003 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.926   Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/home/ext-zorkina-a@ad.speechpro.com/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/ext-zorkina-a@ad.speechpro.com/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [24]:
def chatGPT5():
    
    current = ''
    person1 = ''
    
    while person1 != 'stop':
        
        person1 = input("Person 1: ")
            
        current += f" <Person1> {person1}"
        #print(current)
        
        answer = gen_model.infer(current)
        ans = answer[0][len(current):].split('<')[1].split('>')[1]

        current += f" <Person2> {ans}"
        
        if person1 != 'stop': print("Person 2:", ans)

In [60]:
chatGPT5()

Person 1: Привет!
Person 2:  Привет  
Person 1: Как дела?
Person 2:  Хорошо  
Person 1: Чем сейчас занимаешься ?
Person 2:  Я работаю в банке  
Person 1: Круто! А я лабу делаю(
Person 2:  Я тоже  
Person 1: stop


In [63]:
chatGPT5()

Person 1: Какие твои любимые породы собак?
Person 2:  Собак люблю, но не все, а только кошек  
Person 1: stop
